<a href="https://colab.research.google.com/github/sueta1995/CourseWork_TRPS/blob/machine_learning/classification/odonata_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pickle
import matplotlib.pyplot as plt
import tensorflow
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.initializers import RandomNormal, Constant
from keras import optimizers
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.optimizers import SGD

ModuleNotFoundError: No module named 'tensorflow.keras.layers.convolutional'

In [ ]:
IMG_SIZE = 227
CHANNELS = 3
TRAIN_DIR = 'D:/odonata_data/train_data'
VALID_DIR = 'D:/odonata_data/valid_data'
EPOCHS = 100
BATCH_SIZE = 32

In [22]:
# Для google colab
# from google.colab import drive
# drive.mount('/content/drive')

# VALID_DIR = '/content/drive/MyDrive/odonata_data/valid_data'
# TRAIN_DIR = '/content/drive/MyDrive/odonata_data/train_data'

In [23]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)

valid_dataden = ImageDataGenerator(
    rescale=1./255.
)

In [24]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='rgb',
    batch_size=32,
    seed=1,
    shuffle=True,
    class_mode='categorical'
)

valid_generator = train_datagen.flow_from_directory(
    VALID_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode='rgb',
    batch_size=32,
    seed=7,
    shuffle=True,
    class_mode='categorical'
)

train_num = train_generator.samples
valid_num = valid_generator.samples

Found 18900 images belonging to 54 classes.
Found 2700 images belonging to 54 classes.


In [26]:
model = Sequential()

model.add(Conv2D(filters=96, kernel_size=(11, 11),
                 strides=(4, 4), activation="relu",
                 input_shape=(227, 227, 3)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3), strides= (2, 2)))
model.add(Conv2D(filters=256, kernel_size=(5, 5),
                 strides=(1, 1), activation="relu",
                 padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Conv2D(filters=384, kernel_size=(3, 3),
                 strides=(1, 1), activation="relu",
                 padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(filters=384, kernel_size=(3, 3),
                 strides=(1, 1), activation="relu",
                 padding="same"))
model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=(3, 3),
                 strides=(1, 1), activation="relu",
                 padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(3, 3), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(4096, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(54, activation="softmax"))

model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.001),
              metrics=['accuracy'])

model.summary()

AttributeError: module 'tensorflow' has no attribute 'get_default_graph'

In [ ]:
history = model.fit(
    train_generator,
#     steps_per_epoch=train_num // BATCH_SIZE,
    validation_data=valid_generator,
#     validation_steps=valid_num // BATCH_SIZE,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1
)

Epoch 1/100
268/591 [============>.................] - ETA: 3:12 - loss: 5.3312 - accuracy: 0.0269

In [ ]:
model.save('D:/models/odonata_model')

In [ ]:
# Visualize history
# Plot history: Loss
plt.plot(history.history['val_loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

In [ ]:
# Plot history: Accuracy
plt.plot(history.history['val_accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()